

<h1 align=center><font size = 5>Build a regression Model in Keras By Javier Navarro</font></h1>

<a id="item31"></a>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [2]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's download the data and read it into a <em>pandas</em> dataframe.

In [3]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>

Let's do a quick sanity check of the predictors and the target dataframes.

In [6]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [8]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>

<a id='item32'></a>

## Import Keras

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

#### Let's go ahead and import the Keras library

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model. 

1. One hidden layer of 10 nodes, and a ReLU activation function
2. Use the adam optimizer and the mean squared error as the loss function.

In [19]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.

<a id="item4"></a>

<a id='item34'></a>

## Train and Test the Network

Let's call the function now to create our model.

In [15]:
# build the model
model = regression_model()

Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [20]:
# fit the model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)
model.fit(X_train, y_train, epochs=50, verbose=1)

Epoch 1/50
721/721 [==============================] - 0s 366us/step - loss: 279.0617
Epoch 2/50
721/721 [==============================] - 0s 365us/step - loss: 264.0449
Epoch 3/50
721/721 [==============================] - 0s 327us/step - loss: 252.0500
Epoch 4/50
721/721 [==============================] - 0s 303us/step - loss: 240.6431
Epoch 5/50
721/721 [==============================] - 0s 304us/step - loss: 230.5532
Epoch 6/50
721/721 [==============================] - 0s 331us/step - loss: 221.2180
Epoch 7/50
721/721 [==============================] - 0s 386us/step - loss: 212.9686
Epoch 8/50
721/721 [==============================] - 0s 469us/step - loss: 206.5447
Epoch 9/50
721/721 [==============================] - 0s 278us/step - loss: 199.5022
Epoch 10/50
721/721 [==============================] - 0s 473us/step - loss: 195.0261
Epoch 11/50
721/721 [==============================] - ETA: 0s - loss: 187.102 - 0s 279us/step - loss: 184.6925
Epoch 12/50
721/721 [================

Now we compare results between train and test sets

In [22]:
model.evaluate(X_test, y_test)

309/309 [==============================] - 0s 147us/step


118.67909018507281

Report the mean and the standard deviation of the mean squared errors.

In [23]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

256.2454867285726 0.0


In [25]:
mean_squared_errors = []
for i in range(0, 50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("Mean Squeared Error for a iteration "+str(i+1)+" is: "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

Mean Squeared Error for a iteration 1 is: 104.4261044128813
Mean Squeared Error for a iteration 2 is: 127.70111948155277
Mean Squeared Error for a iteration 3 is: 111.41159837840058
Mean Squeared Error for a iteration 4 is: 116.05120254565983
Mean Squeared Error for a iteration 5 is: 120.94071654285814
Mean Squeared Error for a iteration 6 is: 108.2268778483073
Mean Squeared Error for a iteration 7 is: 131.05563374244664
Mean Squeared Error for a iteration 8 is: 106.37749026894184
Mean Squeared Error for a iteration 9 is: 109.9184279951077
Mean Squeared Error for a iteration 10 is: 120.18513930731221
Mean Squeared Error for a iteration 11 is: 94.38794481098459
Mean Squeared Error for a iteration 12 is: 100.22312586515852
Mean Squeared Error for a iteration 13 is: 112.06934808527382
Mean Squeared Error for a iteration 14 is: 110.80084159381953
Mean Squeared Error for a iteration 15 is: 106.20455364733452
Mean Squeared Error for a iteration 16 is: 106.52347070885322
Mean Squeared Error f